In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from homeloans.tables import Application,PreviousApplication
from homeloans.tables import get_session
from homeloans.tables import initialise_database

In [2]:
db_fn = "H:\\kaggle\\home_loans\\db\\home_loans.db"
initialise_database(db_fn, drop=False)
session = get_session()

2018-07-02 02:05:23,672 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-02 02:05:23,677 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 02:05:23,679 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-02 02:05:23,680 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 02:05:23,681 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Bureau")
2018-07-02 02:05:23,682 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 02:05:23,686 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("PreviousApplication")
2018-07-02 02:05:23,687 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 02:05:23,689 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("CashBalance")
2018-07-02 02:05:23,690 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 02:05:23,691 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Application")
2018-07-02 02:05:23,692 INFO sqlalchemy.engine.base.Engine ()
2018-07-02 02:05:23

In [3]:
qry = session.query(Application)

In [6]:
for entry in qry:
    my_entry = entry
    break

2018-07-02 02:06:41,944 INFO sqlalchemy.engine.base.Engine SELECT "Application".sk_id_curr AS "Application_sk_id_curr", "Application".target AS "Application_target", "Application".name_contract_type AS "Application_name_contract_type", "Application".code_gender AS "Application_code_gender", "Application".flag_own_car AS "Application_flag_own_car", "Application".flag_own_realty AS "Application_flag_own_realty", "Application".cnt_children AS "Application_cnt_children", "Application".amt_income_total AS "Application_amt_income_total", "Application".amt_credit AS "Application_amt_credit", "Application".amt_annuity AS "Application_amt_annuity", "Application".amt_goods_price AS "Application_amt_goods_price", "Application".name_type_suite AS "Application_name_type_suite", "Application".name_income_type AS "Application_name_income_type", "Application".name_education_type AS "Application_name_education_type", "Application".name_family_status AS "Application_name_family_status", "Application".na

In [20]:
[(x.months_balance, x.status) for x in my_entry.bureau_entries[0].balances]

[(-28, 'X'),
 (-27, 'X'),
 (-26, 'X'),
 (-25, 'X'),
 (-24, 'X'),
 (-23, '0'),
 (-22, '0'),
 (-21, 'X'),
 (-20, 'X'),
 (-19, '0'),
 (-18, 'C'),
 (-17, 'C'),
 (-16, 'C'),
 (-15, 'C'),
 (-14, 'C'),
 (-13, 'C'),
 (-12, 'C'),
 (-11, 'C'),
 (-10, 'C'),
 (-9, 'C'),
 (-8, 'C'),
 (-7, 'C'),
 (-6, 'C'),
 (-5, 'C'),
 (-4, 'C'),
 (-3, 'C'),
 (-2, 'C'),
 (-1, 'C'),
 (0, 'C')]

In [24]:
df = pd.read_sql(qry.statement, qry.session.bind)

2018-07-02 02:15:52,920 INFO sqlalchemy.engine.base.Engine SELECT "Application".sk_id_curr, "Application".target, "Application".name_contract_type, "Application".code_gender, "Application".flag_own_car, "Application".flag_own_realty, "Application".cnt_children, "Application".amt_income_total, "Application".amt_credit, "Application".amt_annuity, "Application".amt_goods_price, "Application".name_type_suite, "Application".name_income_type, "Application".name_education_type, "Application".name_family_status, "Application".name_housing_type, "Application".region_population_relative, "Application".days_birth, "Application".days_employed, "Application".days_registration, "Application".days_id_publish, "Application".own_car_age, "Application".flag_mobil, "Application".flag_emp_phone, "Application".flag_work_phone, "Application".flag_cont_mobile, "Application".flag_phone, "Application".flag_email, "Application".occupation_type, "Application".cnt_fam_members, "Application".region_rating_client, 

In [71]:
training = df[~df.target.isnull()]

In [72]:
for col in training:
    if(training.dtypes[col] == "object"):
        print(col)
        training[col] = training[col].astype("category")
    if(training.dtypes[col] == "float64"):
        training[col] = training[col].astype("float32")

F:\envs\homeloans\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


name_contract_type


F:\envs\homeloans\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


code_gender
name_type_suite
name_income_type
name_education_type
name_family_status
name_housing_type
occupation_type
weekday_appr_process_start
organization_type
fondkapremont_mode
housetype_mode
wallsmaterial_mode
emergencystate_mode


In [55]:
tr_dummies = pd.get_dummies(training, drop_first=True)

In [56]:
tr_labels = pd.Series(tr_dummies.target)

In [74]:
#tr_dummies = tr_dummies.drop(["sk_id_curr", "target"], axis=1)
tr_nodummy = training.drop(["sk_id_curr", "target"], axis=1)

In [81]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tr_nodummy, tr_labels, test_size=0.1, random_state=42, stratify=tr_labels)


print('Start training...')
# train
gbm = lgb.LGBMClassifier(task="train", num_leaves=10,
                         learning_rate=0.1,
                         n_estimators=209)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='binary_logloss',
        early_stopping_rounds=5)

print('Start predicting...')
# predict
y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration_)
# eval
print('AUC of prediction:', roc_auc_score(y_train, y_pred))

# feature importances
print('Feature importances:', list(gbm.feature_importances_))


Start training...
[1]	valid_0's binary_logloss: 0.276137
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.272914
[3]	valid_0's binary_logloss: 0.270198
[4]	valid_0's binary_logloss: 0.268089
[5]	valid_0's binary_logloss: 0.266343
[6]	valid_0's binary_logloss: 0.264736
[7]	valid_0's binary_logloss: 0.263389
[8]	valid_0's binary_logloss: 0.262278
[9]	valid_0's binary_logloss: 0.261271
[10]	valid_0's binary_logloss: 0.260341
[11]	valid_0's binary_logloss: 0.2595
[12]	valid_0's binary_logloss: 0.25872
[13]	valid_0's binary_logloss: 0.258039
[14]	valid_0's binary_logloss: 0.257334
[15]	valid_0's binary_logloss: 0.25662
[16]	valid_0's binary_logloss: 0.255944
[17]	valid_0's binary_logloss: 0.255378
[18]	valid_0's binary_logloss: 0.254895
[19]	valid_0's binary_logloss: 0.254442
[20]	valid_0's binary_logloss: 0.254046
[21]	valid_0's binary_logloss: 0.253698
[22]	valid_0's binary_logloss: 0.253338
[23]	valid_0's binary_logloss: 0.253044
[24]	valid_0's

F:\envs\homeloans\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [51]:
tr_labels

1         1.0
2         0.0
3         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
28        0.0
29        0.0
30        1.0
31        0.0
32        0.0
33        0.0
         ... 
356224    1.0
356225    0.0
356226    0.0
356227    0.0
356228    0.0
356229    0.0
356230    0.0
356231    0.0
356232    1.0
356233    0.0
356234    0.0
356235    0.0
356236    0.0
356237    0.0
356238    0.0
356239    0.0
356240    0.0
356241    0.0
356242    0.0
356243    0.0
356244    0.0
356245    0.0
356246    0.0
356247    0.0
356248    0.0
356250    0.0
356251    0.0
356252    0.0
356253    1.0
356254    0.0
Name: target, Length: 307511, dtype: float64